In [ ]:
!pip install tensorflow_text

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import pandas as pd
import csv

from google.colab import drive
drive.mount("/content/drive")

In [ ]:
def tf_lower_and_split_punct(text):
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text, '[^ a-z.?!,]', '')
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text, '[.?!,]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)
  return text

In [ ]:
def create_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="sentences")
  preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", name="preprosesing")(text_input)
  encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",trainable=False, name="embedding")(preprocessor)
  # pooled_output = outputs["pooled_output"]# [batch_size, 768].
  # sequence_output = outputs["sequence_output"]# [batch_size, seq_length, 768].
  x1 = tf.keras.layers.Dense(1024, activation="relu")(encoder["pooled_output"])
  x2 = tf.keras.layers.Dropout(0.1)(x1)
  x3 = tf.keras.layers.Dense(1, activation ="sigmoid", name="output_layer")(x2)

  model = tf.keras.Model(inputs=[text_input], outputs=[x3], name="Binary_Classification_Model")

  model.compile(
      optimizer="adam",
      loss = tf.keras.losses.BinaryCrossentropy(),
      metrics=["accuracy"]
  )

  model.summary()
  return model

with open("/content/drive/MyDrive/IOH/spam_or_not/data/emails.csv") as f:
  csvreader = csv.reader(f, delimiter=",")
  next(csvreader)
  masukan = []
  keluaran = []
  for a, i in enumerate(csvreader):
      inp1 = tf_lower_and_split_punct(i[0]).numpy().decode()
      inp2 = tf_lower_and_split_punct(i[2]).numpy().decode()
      tar = int(i[1])
      masukan.append(inp1)
      masukan.append(inp2)
      keluaran.append(int(tar))
      keluaran.append(int(tar))

BUFFER_SIZE = len(masukan)
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((masukan, keluaran)).shuffle(BUFFER_SIZE)
train = dataset.take(int(0.7*len(masukan)))
remaining = dataset.skip(int(0.7*len(masukan)))
validation = remaining.take(int(0.15*len(masukan)))
test = remaining.skip(int(0.15*len(masukan)))

train = train.batch(BATCH_SIZE)
validation = validation.batch(BATCH_SIZE)

In [ ]:
for example_input_batch, example_target_batch in train.take(1):
  print(example_input_batch[:5])
  print()
  print(example_target_batch[:5])
  break

In [ ]:
model = create_model()
ckpt_path = "/content/drive/MyDrive/IOH/spam_or_not/checkpoint/model.{epoch:02d}-{val_loss:.4f}.h5"
ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath=ckpt_path, 
    save_weights_only=True,
    save_best_only=True,
    verbose=1, 
)

In [ ]:
model.fit(train, validation_data=validation, epochs=15, callbacks=[ckpt])

In [ ]:
teks_spam = "DISNAKERJA and others share their thoughts on LinkedIn"
model.predict([teks_spam])

In [ ]:
teks = [
        "Bangkit Academy led by Google, Tokopedia, Gojek, & Traveloka and others share their thoughts on LinkedIn",
        "Kanao, welcome to Samsung",
        "[educba.com] Angular JS Certification Training Course (9 Courses, 5+ Projects)",
        "PTEB UTS 07111940000001",
        "Lomba Desain IG : itsmeeeaow",
        "Photo from Vanny Ezha",
        "SPCS1_tugas03_07111940000001",
        "Pendaftaran Akun Kampus Merdeka",
]
model.predict(teks)

In [ ]:
benar = []
for tes,lab in test.take(100):
  # print(tes.numpy().decode())
  if sum(model.predict([tes.numpy().decode()])) >= 0.5 and lab.numpy() == 1:
    benar.append(1)
  elif sum(model.predict([tes.numpy().decode()])) <= 0.5 and lab.numpy() == 1:
    pass
  elif sum(model.predict([tes.numpy().decode()])) >= 0.5 and lab.numpy() == 0:
    # print(False)
    pass
  elif sum(model.predict([tes.numpy().decode()])) <= 0.5 and lab.numpy() == 0:
    # print(True)
    benar.append(1)

sum(benar)
  # print(model.predict([tes.numpy().decode()]))
  # print(lab.numpy())

In [ ]:
model.save("/content/drive/MyDrive/IOH/spam_or_not/saved_model")

In [ ]:
import pickle

with open("/content/drive/MyDrive/IOH/spam_or_not/pickle/model_spam.pkl","wb") as f:
  pickle.dump(model, f)

## Keterangan : lebih dari 0.5 berpotensi spam, kurang dari tersebut tidak berpotensi spam

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/IOH/spam_or_not/saved_model/")

In [ ]:
teks = [
        "Bangkit Academy led by Google, Tokopedia, Gojek, & Traveloka and others share their thoughts on LinkedIn",
        "Kanao, welcome to Samsung",
        "[educba.com] Angular JS Certification Training Course (9 Courses, 5+ Projects)",
        "PTEB UTS 07111940000001",
        "Lomba Desain IG : itsmeeeaow",
        "Photo from Vanny Ezha",
        "SPCS1_tugas03_07111940000001",
        "Pendaftaran Akun Kampus Merdeka",
]

model(tf.constant(teks))